In [ ]:
from typing import Tuple
import pandas as pd
import numpy as np

from pathlib import Path
from matplotlib import pyplot, image

In [ ]:
def returnAnnotations(file_attributos, file_annotations):
  df_attributes = pd.read_csv(file_attributos, header = None)
  list_attributes = list()
  list_attributes.append(df_attributes[0].values)
  handle = open(file_annotations)
  scores = dict()
  confidence = dict()
  for line in handle:
      words = line.split()
      i=0
      for word in words:
          if i == 0:
              scores[word]=list()
              confidence[word] = list()
              i+=1
          else:
              values = word.split(',')
              scores[words[0]].append(float(values[0]))
              confidence[words[0]].append(float(values[1]))
              i+=1
  scores = pd.DataFrame(scores).T
  scores.columns = list_attributes
  confidence = pd.DataFrame(confidence).T
  confidence.columns = list_attributes
  return scores,confidence


In [ ]:
file_attributos = '../../data/annotations/attributes.txt'
file_annotations = '../../data/annotations/annotations.tsv'
file_training = '../../data/training_test_splits/holdout_split/training.txt'
file_test = '../../data/training_test_splits/holdout_split/test.txt'
def returnTrainingTest(file_attributos, file_annotations, file_training, file_test):
  scores,confidence = returnAnnotations(file_attributos, file_annotations)
  df_training = pd.read_csv(file_training, header = None)
  df_test = pd.read_csv(file_test, header = None)
  scores_training = scores[scores.index.isin(df_training[0].values)]
  scores_test = scores[scores.index.isin(df_test[0].values)]
  confidence_training = confidence[confidence.index.isin(df_training[0].values)]
  confidence_test = confidence[confidence.index.isin(df_test[0].values)]
  return scores_training, scores_test, confidence_training, confidence_test

In [ ]:
scores_training, scores_test, confidence_training, confidence_test = returnTrainingTest(file_attributos, file_annotations, file_training, file_test)

In [ ]:
scores_training

In [ ]:
scores_test

In [ ]:
confidence_training

In [ ]:
confidence_test

In [ ]:
confidence_training = confidence_training.apply(lambda x: [0 if y <= 5 else 1 for y in x])
confidence_training

In [ ]:
confidence_test = confidence_test.apply(lambda x: [0 if y <= 5 else 1 for y in x])
confidence_test

In [ ]:
#df_training = pd.DataFrame(scores_training*confidence_training, columns=scores_training.columns, index=scores_training.index)
df_training = scores_training
df_training.columns = df_training.columns.get_level_values(0)
df_training

In [ ]:
#df_test = pd.DataFrame(scores_test*confidence_test, columns=scores_test.columns, index=scores_test.index)
df_test = scores_test
df_test.columns = df_test.columns.get_level_values(0)
df_test

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display_markdown
display_markdown('# Verificação visual do balanço do banco de dados no conjunto de treino.',raw=True)
plt.figure(figsize=(70, 4))
plt.title('Frequencia das classes no conjunto de treino')
palette = sns.color_palette('gist_rainbow', len(df_training.columns.values))
sns.barplot(df_training.columns.values, df_training.sum().values,palette=palette)

In [ ]:
path_img = Path('../../data/ignored/images/imageLD/')
import os
def returnPathImages():
    imageLD = {}
    datasets = os.listdir(path_img)
    for dataset in datasets:
        imgs_name = os.listdir(path_img/Path(dataset))
        for name in imgs_name:
            img_file = path_img/Path(dataset)/Path(name)
            imageLD[str(Path(dataset)/Path(name))]=img_file
    return imageLD

In [ ]:
images_paths = returnPathImages()

In [ ]:
images_paths

In [ ]:
df_training_image = df_training[df_training.index.isin(images_paths.keys())]
df_training_image

In [ ]:
df_test_image = df_test[df_test.index.isin(images_paths.keys())]
df_test_image

In [ ]:
iter_img = iter(images_paths.items())

In [ ]:
a_name, a_img = next(iter_img)

In [ ]:
image.imread(a_img)

In [ ]:
a_name

In [ ]:
df_training.loc[a_name]

In [ ]:
images_paths_train = {k: v for k, v in images_paths.items() if k in df_training.index}

In [ ]:
images_paths_teste = {k: v for k, v in images_paths.items() if k in df_test.index}

In [ ]:
data_treino = df_training.copy()
data_treino['path'] = df_training.index

In [ ]:
data_teste = df_test.copy()
data_teste['path'] = df_test.index

In [ ]:
data_treino

In [ ]:
data_treino.columns[:-1]

In [ ]:
data_treino.columns[-1]

In [ ]:
img_height = 224
img_width = 224

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
datagen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)

In [ ]:
train_generator = datagen.flow_from_dataframe(dataframe=data_treino, directory=path_img,
                                              target_size=(img_height, img_width),
                                              x_col='path',
                                              y_col=data_treino.columns[:-1],
                                              class_mode="raw",
                                              subset='training',
                                              shuffle=True,
                                              seed=7)

validation_generator = datagen.flow_from_dataframe(dataframe=data_treino, directory=path_img,
                                             target_size=(img_height, img_width),
                                             x_col='path',
                                             y_col=data_treino.columns[:-1],
                                             class_mode="raw",
                                             subset='validation',
                                             shuffle=True,
                                             seed=7)

test_generator = datagen.flow_from_dataframe(dataframe=data_teste, directory=path_img,
                                             target_size=(img_height, img_width),
                                             x_col='path',
                                             y_col=data_treino.columns[:-1],
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=7)

In [ ]:
model_feature_extraction = VGG16(weights='imagenet', include_top=False)

In [ ]:
train_features = model_feature_extraction.predict(train_generator)

In [ ]:
validation_features = model_feature_extraction.predict(validation_generator)

In [ ]:
test_features = model_feature_extraction.predict(test_generator)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
img_height = 224
img_width = 224
pre_trained_model  = VGG16(include_top=False, input_shape=(img_height,img_width,3), weights='imagenet')
transfer_layer = pre_trained_model.get_layer('block5_pool')
conv_model = Model(inputs=pre_trained_model.input, outputs=transfer_layer.output)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
import cv2

In [ ]:
n_imgs = 100
X_train = []
y_train = []
for i, (img_name, img_path) in enumerate(images_paths_train.items()):
    if i >= n_imgs: break
    x = image.imread(img_path)
    x = cv2.resize(x, (400,400))
    x = x.flatten()
    print(len(x))
    X_train.append(x)
    y_train.append(df_training.loc[img_name].values)

In [ ]:
n_imgs = 100
X_teste = []
y_teste = []
for i, (img_name, img_path) in enumerate(images_paths_teste.items()):
    if i >= n_imgs: break
    x = image.imread(img_path)
    x = cv2.resize(x, (400,400))
    x = x.flatten()
    print(len(x))
    X_teste.append(x)
    y_teste.append(df_test.loc[img_name].values)

In [ ]:
clf = MultiOutputRegressor(SVR())

In [ ]:
clf.fit(X_train, y_train)


In [ ]:
X_train

In [ ]:
y_train

In [ ]:
y_predicted = clf.predict(X_teste)

In [ ]:
y_predicted

In [ ]:
np.square(y_teste - y_predicted).mean()

In [ ]:

import numpy as np
def visualizarAmostras(numero_exemplos, df_path, df_training):
  rango = np.arange(df_training.shape[0])
  amostras = np.random.choice(a = rango, size = numero_exemplos, replace = False)
  for amostra in amostras:
    visualizeImageByIndex(amostra,df_path, df_training)

In [ ]:
display_markdown('# Verificação visual de algumas amostras do conjunto de treino imageLD',raw=True)
visualizarAmostras(numero_exemplos=5, df_path= imageLD, df_training= df_training_imageLD)

In [ ]:
display_markdown('# Verificação visual de algumas amostras do conjunto de treino imageAlignedLD',raw=True)
visualizarAmostras(numero_exemplos=5, df_path= imageAlignedLD, df_training= df_training_imageAlignedLD)

In [ ]:
def returnImageByIndex(i, df_path, df_training):
  img = cv2.imread(df_path[df_path.index==df_training.index[i]].values[0][0])
  return img

In [ ]:
Path_imageLD = imageLD[imageLD.index.isin(df_training_imageLD.index)]
Path_imageLD = pd.concat([Path_imageLD, df_training_imageLD], axis=1)
Path_imageAlignedLD = imageAlignedLD[imageAlignedLD.index.isin(df_training_imageAlignedLD.index)]
Path_imageAlignedLD = pd.concat([Path_imageAlignedLD, df_training_imageAlignedLD], axis=1)
data_treino = pd.concat([Path_imageLD, Path_imageAlignedLD], axis=0)
data_treino

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
img_height = 224
img_width = 224
pre_trained_model  = VGG16(include_top=False, input_shape=(img_height,img_width,3), weights='imagenet')
transfer_layer = pre_trained_model.get_layer('block5_pool')
conv_model = Model(inputs=pre_trained_model.input, outputs=transfer_layer.output)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
# Start a new Keras Sequential model.
model = Sequential()

# Add the convolutional part of the VGG16 model from above.
model.add(conv_model)
model.add(BatchNormalization())
# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.1))

# Add the final layer for the actual classification.
labels = 40
model.add(Dense(labels, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes = True)

In [ ]:
input_shape = pre_trained_model.layers[0].output_shape[0][1:3]
input_shape

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 20
datagen = ImageDataGenerator(rescale=1/255., validation_split=0.2)
train_generator = datagen.flow_from_dataframe(dataframe=data_treino, directory=path_drive,
                                             x_col='Path',
                                             y_col=data_treino.columns[1:],
                                             target_size=(img_height,img_width),
                                             batch_size=BATCH_SIZE,
                                             class_mode="raw",
                                             subset='training',
                                             shuffle=True,
                                             seed=7)

validation_generator = datagen.flow_from_dataframe(dataframe=data_treino, directory=path_drive,
                                             x_col='Path',
                                             y_col=data_treino.columns[1:],
                                             target_size=(img_height,img_width),
                                             batch_size=1,
                                             class_mode="raw",
                                             subset='validation',
                                             shuffle=True,
                                             seed=7)

In [ ]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
  SS_res =  K.sum(K.square( y_true-y_pred ))
  SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
  return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
model.compile(optimizer = optimizer, loss='mean_absolute_error', metrics=[coeff_determination])

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

In [ ]:
from keras import utils, callbacks
from tensorflow.keras.callbacks import LearningRateScheduler
earlystopping = callbacks.EarlyStopping(monitor='val_loss', mode='min', patience= 15, restore_best_weights=True)
epochs = 250

In [ ]:
total_train = 6368
total_val = 1592
history = model.fit(train_generator, epochs=epochs, steps_per_epoch=total_train//BATCH_SIZE, validation_data=validation_generator, validation_steps=total_val, verbose=1, callbacks=[LearningRateScheduler(lr_schedule),earlystopping])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b--', label = 'MAE de Treino')
plt.plot(history.history['val_loss'], 'r-.', label = 'MAE de Validação')
plt.xlabel('Epochs')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['coeff_determination'], 'b--', label = 'R2 de Treino')
plt.plot(history.history['val_coeff_determination'], 'r-.', label = 'R2 de Validação')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
model.save_weights('weights.h5')

In [ ]:
test_imageLD = imageLD[imageLD.index.isin(df_test_imageLD.index)]
test_imageLD = pd.concat([test_imageLD, df_test_imageLD], axis=1)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(dataframe=test_imageLD, directory=path_drive, x_col='Path',
                                                  y_col=test_imageLD.columns[1:],
                                                  target_size=(img_height,img_width),
                                                  batch_size=1,
                                                  class_mode="raw",
                                                  shuffle=False)

In [ ]:
predictions  = model.predict(test_generator)

In [ ]:
predictions.shape

In [ ]:
y = df_test_imageLD.values

In [ ]:
from numpy import linalg as LA
MSE = LA.norm((y-predictions), 2) ** 2/y.size

In [ ]:
MSE #Resultado teste usando o banco imageLD e Holdout

In [ ]:
from sklearn.metrics import average_precision_score
average_precision_score(np.where(0.8 > y, 1, 0), predictions) #Resultado teste usando o banco imageLD e Holdout

In [ ]:
test_imageAlignedLD = imageAlignedLD[imageAlignedLD.index.isin(df_test_imageAlignedLD.index)]
test_imageAlignedLD = pd.concat([test_imageAlignedLD, df_test_imageAlignedLD], axis=1)

In [ ]:
test_generator2 = test_datagen.flow_from_dataframe(dataframe=test_imageAlignedLD, directory=path_drive, x_col='Path',
                                                  y_col=test_imageAlignedLD.columns[1:],
                                                  target_size=(img_height,img_width),
                                                  batch_size=1,
                                                  class_mode="raw",
                                                  shuffle=False)

In [ ]:
predictions2  = model.predict(test_generator2)

In [ ]:
y2 = df_test_imageAlignedLD.values

In [ ]:
MSE2 = LA.norm((y-predictions), 2) ** 2/y.size
MSE2 #Resultado teste usando o banco imageAlignedLD e Holdout

In [ ]:
average_precision_score(np.where(0.8 > y2, 1, 0), predictions2) #Resultado teste usando o banco imageAlignedLD e Holdout